### Importing libraries and reading data

In [16]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_url, 'lxml')

In [17]:
column_names = ['PostalCode', 'Borough', 'Neighborhood'] 
toronto = pd.DataFrame(columns=column_names)

### Parsing html

In [18]:
tableSoup = soup.find_all("table")
table = tableSoup[0]
rows = table.find_all("tr")
for row in rows:
    columns = row.find_all("td")
    headers = row.find_all("th")
    if len(columns) == 0 : continue
    postcode = columns[0].text
    borough = columns[1].text
    neigh = columns[2].text.split("\n")[0]
    toronto = toronto.append({'PostalCode': postcode,
                          'Borough': borough,
                          'Neighborhood': neigh}, ignore_index=True)
toronto.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


###  Ignore cells with a borough that is Not assigned.

In [19]:
toronto = toronto[toronto['Borough'].str.contains("[^Not assigned]")==True]
toronto.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


### Combining neighborhoods with the same Postal Code into one row with the neighborhoods separated with a comma

In [20]:
toronto = pd.DataFrame({'Neighborhood' : toronto.groupby(by = ['PostalCode', 'Borough'])['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))}).reset_index()
toronto.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Neighborhood not assigned, then the neighborhood will be the same as the borough

In [21]:
toronto['Neighborhood'].replace(to_replace="Not assigned", value=toronto['Borough'], inplace=True)
toronto.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### DataFrame shape

In [22]:
toronto.shape

(103, 3)

### Reading geospatial data

In [23]:
!wget -q -o 'Geospatial_data.csv' http://cocl.us/Geospatial_data
geoDF = pd.read_csv("Geospatial_data")
geoDF.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Concatenate Latitude and Longitude to the appropriate Postal code

In [24]:
toronto = pd.concat([toronto, geoDF], axis=1)
toronto = toronto.drop('Postal Code', axis =1)
toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Exploring Toronto (Downtown, East, West, Central) only

In [25]:
toronto = toronto[toronto['Borough'].str.contains("Toronto")==True]
toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


### Finding near by venues

In [26]:
CLIENT_ID = 'S0S0W4P24DNIDRQL35RZPRULG5WKL2WQE2UHZR2VS3HHHBUY' # your Foursquare ID
CLIENT_SECRET = 'WIUPMQLQWAN13SPPU1DQZR04V5S32P1WQWQGZC3FUJYWJKV5'  # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: S0S0W4P24DNIDRQL35RZPRULG5WKL2WQE2UHZR2VS3HHHBUY
CLIENT_SECRET:WIUPMQLQWAN13SPPU1DQZR04V5S32P1WQWQGZC3FUJYWJKV5


In [27]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            100)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        print(name, len(results))
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['PostalCode', 
                  'PostalCode Latitude', 
                  'PostalCode Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [28]:
toronto_venues = getNearbyVenues(names=toronto['PostalCode'],
                                   latitudes=toronto['Latitude'],
                                   longitudes=toronto['Longitude']
                                  )

M4E
M4E 4
M4K
M4K 42
M4L
M4L 18
M4M
M4M 40
M4N
M4N 5
M4P
M4P 7
M4R
M4R 22
M4S
M4S 36
M4T
M4T 2
M4V
M4V 14
M4W
M4W 4
M4X
M4X 45
M4Y
M4Y 85
M5A
M5A 49
M5B
M5B 100
M5C
M5C 100
M5E
M5E 54
M5G
M5G 84
M5H
M5H 100
M5J
M5J 100
M5K
M5K 100
M5L
M5L 100
M5N
M5N 1
M5P
M5P 4
M5R
M5R 25
M5S
M5S 33
M5T
M5T 98
M5V
M5V 14
M5W
M5W 94
M5X
M5X 100
M6G
M6G 16
M6H
M6H 18
M6J
M6J 64
M6K
M6K 21
M6P
M6P 24
M6R
M6R 16
M6S
M6S 39
M7Y
M7Y 17


In [29]:
toronto_venues.head()

,PostalCode,PostalCode Latitude,PostalCode Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M4E,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
1,M4E,43.676357,-79.293031,Starbucks,43.678798,-79.298045,Coffee Shop
2,M4E,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
3,M4E,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,M4K,43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


### Number of venues

In [30]:
len(toronto_venues['Venue'].unique())

1098

### Number of venue categories

In [84]:
len(toronto_venues['Venue Category'].unique())

60

### Filtering venues to work on restaurants only using keywords such as "Restaurant, Pizza, Burger, etc.."

In [32]:
toronto_venues = toronto_venues[toronto_venues['Venue Category'].str.contains("Restaurant|Pizza|Burger|Diner|Salad|BBQ Joint|Food|Burrito")==True]
toronto_venues.head()

,PostalCode,PostalCode Latitude,PostalCode Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
4,M4K,43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant
7,M4K,43.679557,-79.352188,Messini Authentic Gyros,43.677827,-79.350569,Greek Restaurant
8,M4K,43.679557,-79.352188,Cafe Fiorentina,43.677743,-79.350115,Italian Restaurant
9,M4K,43.679557,-79.352188,Mezes,43.677962,-79.350196,Greek Restaurant
10,M4K,43.679557,-79.352188,Christina's On The Danforth,43.678240,-79.349185,Greek Restaurant


### Number of restaurants in Toronto

In [85]:
len(toronto_venues['Venue'].unique())

363

### Importing map libraries

In [33]:
# import k-means from clustering stage
from sklearn.cluster import KMeans
import requests
import numpy as np

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


### Map of Toronto

In [63]:
map_toronto = folium.Map(location=[43.6532, -79.3832], zoom_start=11)
map_toronto

### Adding restaurant venues as markers to the map of Toronto

In [64]:
# add markers to map
for lat, lng, postal, neighborhood in zip(toronto_venues['Venue Latitude'], toronto_venues['Venue Longitude'], toronto_venues['PostalCode'], toronto_venues['Venue']):
    label = '{}, {}'.format(postal,neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Analyzing restaurants

In [40]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['PostalCode'] = toronto_venues['PostalCode'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,PostalCode,Afghan Restaurant,Airport Food Court,American Restaurant,Arepa Restaurant,Asian Restaurant,BBQ Joint,Belgian Restaurant,Brazilian Restaurant,Burger Joint,...,Restaurant,Salad Place,Seafood Restaurant,Southern / Soul Food Restaurant,Sushi Restaurant,Taiwanese Restaurant,Tapas Restaurant,Thai Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
4,M4K,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,M4K,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,M4K,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,M4K,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,M4K,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Grouping venues with postal code by calculating the frequency using mean

In [42]:
toronto_grouped = toronto_onehot.groupby('PostalCode').mean().reset_index()
toronto_grouped.head()

,PostalCode,Afghan Restaurant,Airport Food Court,American Restaurant,Arepa Restaurant,Asian Restaurant,BBQ Joint,Belgian Restaurant,Brazilian Restaurant,Burger Joint,...,Restaurant,Salad Place,Seafood Restaurant,Southern / Soul Food Restaurant,Sushi Restaurant,Taiwanese Restaurant,Tapas Restaurant,Thai Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,M4K,0.0,0.0,0.052632,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.052632,0.0,0.000000,0.0,0.052632,0.0,0.0,0.0,0.0,0.0
1,M4L,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.166667,...,0.000000,0.0,0.000000,0.0,0.166667,0.0,0.0,0.0,0.0,0.0
2,M4M,0.0,0.0,0.181818,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.090909,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3,M4N,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
4,M4P,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.500000,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


#### Top 5 restaurants in Toronto for each postal code

In [43]:
num_top_venues = 5

for hood in toronto_grouped['PostalCode']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['PostalCode'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----M4K----
                  venue  freq
0      Greek Restaurant  0.47
1    Italian Restaurant  0.11
2  Caribbean Restaurant  0.05
3     Indian Restaurant  0.05
4     Health Food Store  0.05


----M4L----
                  venue  freq
0           Pizza Place  0.17
1      Sushi Restaurant  0.17
2  Fast Food Restaurant  0.17
3         Burrito Place  0.17
4          Burger Joint  0.17


----M4M----
                       venue  freq
0        American Restaurant  0.18
1         Italian Restaurant  0.18
2                      Diner  0.09
3  Latin American Restaurant  0.09
4  Middle Eastern Restaurant  0.09


----M4N----
                 venue  freq
0   Dim Sum Restaurant   1.0
1     Greek Restaurant   0.0
2    Indian Restaurant   0.0
3   Italian Restaurant   0.0
4  Japanese Restaurant   0.0


----M4P----
                     venue  freq
0        Food & Drink Shop   0.5
1             Burger Joint   0.5
2        Afghan Restaurant   0.0
3  New American Restaurant   0.0
4        Indian Restaur

#### Top 10 restaurants for each postal code

In [44]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [49]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['PostalCode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
toronto_venues_sorted = pd.DataFrame(columns=columns)
toronto_venues_sorted['PostalCode'] = toronto_grouped['PostalCode']

for ind in np.arange(toronto_grouped.shape[0]):
    toronto_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

toronto_venues_sorted

,PostalCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4K,Greek Restaurant,Italian Restaurant,Restaurant,Caribbean Restaurant,Pizza Place,Indian Restaurant,Health Food Store,Diner,Sushi Restaurant,American Restaurant
1,M4L,Burger Joint,Sushi Restaurant,Fast Food Restaurant,Italian Restaurant,Pizza Place,Burrito Place,Vietnamese Restaurant,Doner Restaurant,Dumpling Restaurant,Eastern European Restaurant
2,M4M,Italian Restaurant,American Restaurant,Seafood Restaurant,Diner,Chinese Restaurant,New American Restaurant,Latin American Restaurant,Comfort Food Restaurant,Middle Eastern Restaurant,Ethiopian Restaurant
3,M4N,Dim Sum Restaurant,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant
4,M4P,Food & Drink Shop,Burger Joint,Vietnamese Restaurant,Dim Sum Restaurant,Food Truck,Food Court,Food,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant
5,M4R,Fast Food Restaurant,Mexican Restaurant,Chinese Restaurant,Diner,Vietnamese Restaurant,Dim Sum Restaurant,Food Court,Food & Drink Shop,Food,Filipino Restaurant
6,M4S,Pizza Place,Sushi Restaurant,Seafood Restaurant,Italian Restaurant,Greek Restaurant,Thai Restaurant,Diner,Japanese Restaurant,Restaurant,Burger Joint
7,M4V,Vietnamese Restaurant,American Restaurant,Sushi Restaurant,Pizza Place,Ethiopian Restaurant,Diner,Doner Restaurant,Dumpling Restaurant,Eastern European Restaurant,Fast Food Restaurant
8,M4X,Restaurant,Indian Restaurant,Pizza Place,Italian Restaurant,Chinese Restaurant,Diner,Japanese Restaurant,Caribbean Restaurant,Taiwanese Restaurant,Thai Restaurant
9,M4Y,Japanese Restaurant,Burger Joint,Sushi Restaurant,Restaurant,Mediterranean Restaurant,Fast Food Restaurant,Vietnamese Restaurant,American Restaurant,Burrito Place,Caribbean Restaurant


#### Clustering neighborhoods with kmeans

In [76]:
# set number of clusters
kclusters = 3

toronto_grouped_clustering = toronto_grouped.drop('PostalCode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:34]

array([1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [77]:
toronto_merged = toronto[0:34]

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge scarborough_grouped with scarborough_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(toronto_venues_sorted.set_index('PostalCode'), on='PostalCode')

toronto_merged.head() # check the last columns!

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,1,Greek Restaurant,Italian Restaurant,Restaurant,Caribbean Restaurant,Pizza Place,Indian Restaurant,Health Food Store,Diner,Sushi Restaurant,American Restaurant
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,1,Burger Joint,Sushi Restaurant,Fast Food Restaurant,Italian Restaurant,Pizza Place,Burrito Place,Vietnamese Restaurant,Doner Restaurant,Dumpling Restaurant,Eastern European Restaurant
43,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Italian Restaurant,American Restaurant,Seafood Restaurant,Diner,Chinese Restaurant,New American Restaurant,Latin American Restaurant,Comfort Food Restaurant,Middle Eastern Restaurant,Ethiopian Restaurant
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,1,Dim Sum Restaurant,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant


#### Visualize the resuling clusters

In [78]:
import matplotlib.cm as cm
import matplotlib.colors as colors
# create map
map_clusters = folium.Map(location=[43.6532, -79.3832], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['PostalCode'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Cluster 1

In [79]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0]

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
43,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Italian Restaurant,American Restaurant,Seafood Restaurant,Diner,Chinese Restaurant,New American Restaurant,Latin American Restaurant,Comfort Food Restaurant,Middle Eastern Restaurant,Ethiopian Restaurant


#### Cluster 2

In [80]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1]

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,1,Greek Restaurant,Italian Restaurant,Restaurant,Caribbean Restaurant,Pizza Place,Indian Restaurant,Health Food Store,Diner,Sushi Restaurant,American Restaurant
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,1,Burger Joint,Sushi Restaurant,Fast Food Restaurant,Italian Restaurant,Pizza Place,Burrito Place,Vietnamese Restaurant,Doner Restaurant,Dumpling Restaurant,Eastern European Restaurant
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,1,Dim Sum Restaurant,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant
45,M4P,Central Toronto,Davisville North,43.712751,-79.390197,1,Food & Drink Shop,Burger Joint,Vietnamese Restaurant,Dim Sum Restaurant,Food Truck,Food Court,Food,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant
46,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,1,Fast Food Restaurant,Mexican Restaurant,Chinese Restaurant,Diner,Vietnamese Restaurant,Dim Sum Restaurant,Food Court,Food & Drink Shop,Food,Filipino Restaurant
47,M4S,Central Toronto,Davisville,43.704324,-79.388790,1,Pizza Place,Sushi Restaurant,Seafood Restaurant,Italian Restaurant,Greek Restaurant,Thai Restaurant,Diner,Japanese Restaurant,Restaurant,Burger Joint
48,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049,1,Vietnamese Restaurant,American Restaurant,Sushi Restaurant,Pizza Place,Ethiopian Restaurant,Diner,Doner Restaurant,Dumpling Restaurant,Eastern European Restaurant,Fast Food Restaurant
50,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Cluster 3

In [81]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2]

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
64,M5P,Central Toronto,"Forest Hill North, Forest Hill West",43.696948,-79.411307,2,Sushi Restaurant,Mexican Restaurant,Vietnamese Restaurant,Cuban Restaurant,Food Court,Food & Drink Shop,Food,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant
